In [17]:
from easydict import EasyDict as edict
from dataset import get_dataset
from torch.utils.data import DataLoader

data_args = edict(
    split="train",
    task="vqa",
    dataset="MedXpertQA",
    seed=0,
    image_path="/media/yesindeed/DATADRIVE1/mount/remote_cse/datasets/MedXpertQA",
)

dataset = get_dataset(data_args)
data_loader = DataLoader(dataset, batch_size=1,
                         collate_fn=lambda batch: batch[0])

Logger is not set.


In [18]:
dataset[0]

{'image': [tensor([[[  0,   0,   0,  ...,  21,  17,  18],
           [  0,   0,   0,  ...,  23,  19,  19],
           [  0,   0,   0,  ...,  18,  16,  12],
           ...,
           [  0,   0,   0,  ..., 132, 125, 143],
           [  0,   0,   0,  ..., 140, 133, 146],
           [  0,   0,   0,  ..., 155, 147, 164]],
  
          [[  0,   0,   0,  ...,  21,  17,  18],
           [  0,   0,   0,  ...,  23,  19,  19],
           [  0,   0,   0,  ...,  18,  16,  12],
           ...,
           [  0,   0,   0,  ..., 132, 125, 143],
           [  0,   0,   0,  ..., 140, 133, 146],
           [  0,   0,   0,  ..., 155, 147, 164]],
  
          [[  0,   0,   0,  ...,  21,  17,  18],
           [  0,   0,   0,  ...,  23,  19,  19],
           [  0,   0,   0,  ...,  18,  16,  12],
           ...,
           [  0,   0,   0,  ..., 132, 125, 143],
           [  0,   0,   0,  ..., 140, 133, 146],
           [  0,   0,   0,  ..., 155, 147, 164]]], dtype=torch.uint8),
  tensor([[[114, 102, 101,  ...

In [19]:
for batch in data_loader:
    break

batch["image"]

[tensor([[[  0,   0,   0,  ...,  21,  17,  18],
          [  0,   0,   0,  ...,  23,  19,  19],
          [  0,   0,   0,  ...,  18,  16,  12],
          ...,
          [  0,   0,   0,  ..., 132, 125, 143],
          [  0,   0,   0,  ..., 140, 133, 146],
          [  0,   0,   0,  ..., 155, 147, 164]],
 
         [[  0,   0,   0,  ...,  21,  17,  18],
          [  0,   0,   0,  ...,  23,  19,  19],
          [  0,   0,   0,  ...,  18,  16,  12],
          ...,
          [  0,   0,   0,  ..., 132, 125, 143],
          [  0,   0,   0,  ..., 140, 133, 146],
          [  0,   0,   0,  ..., 155, 147, 164]],
 
         [[  0,   0,   0,  ...,  21,  17,  18],
          [  0,   0,   0,  ...,  23,  19,  19],
          [  0,   0,   0,  ...,  18,  16,  12],
          ...,
          [  0,   0,   0,  ..., 132, 125, 143],
          [  0,   0,   0,  ..., 140, 133, 146],
          [  0,   0,   0,  ..., 155, 147, 164]]], dtype=torch.uint8),
 tensor([[[114, 102, 101,  ..., 170, 175, 172],
          [114,

In [20]:
type(batch["image"]) == list

True

In [21]:
import re


def extract_choice(text):
    # looks for patterns like: (A), A), answer is A, answer: A, etc.
    match = re.search(r"\b([A-D])\b(?=[).: ]?)", text)
    return match.group(1) if match else None

In [22]:
text = "Here's the breakdown of the case and why the answer is **(E)**:\n\n**Understanding the Case**\n\n*   **Clinical Presentation:** The patient presents with concerning symptoms: chest pain, neck lump, drooping eyelid/smaller eye, miotic pupil (suggesting cranial nerve involvement), wheezing, and cervical lymphadenopathy.\n*   **Imaging:** The MRI shows a posterior mediastinal mass.\n*   **Biopsy:** The biopsy reveals a specific type of tumor:\n    *   Spindle-shaped cells with moderate cytoplasm and small nuclei.\n    *   Scattered mature ganglion cells.\n    *   Immunohistochemical markers: S-100, synaptophysin, chromogranin, and LCA.  These markers are highly suggestive of a neuroblastoma.\n\n**Neuroblastoma and Prognosis**\n\nNeuroblastoma is a cancer that arises from immature nerve cells. It's most common in young children. The prognosis is heavily influenced by"

extract_choice(text)

In [23]:
import difflib


def get_content_between_a_b(start_tag, end_tag, text):
    extracted_text = ""
    start_index = text.find(start_tag)
    while start_index != -1:
        end_index = text.find(end_tag, start_index + len(start_tag))
        if end_index != -1:
            extracted_text += text[start_index + len(start_tag) : end_index] + " "
            start_index = text.find(start_tag, end_index + len(end_tag))
        else:
            break

    return extracted_text.strip()


def extract(text, type, hard=True):
    if text:
        target_str = get_content_between_a_b(f"<{type}>", f"</{type}>", text)
        if target_str:
            return target_str
        elif hard:
            return text
        else:
            return ""
    else:
        return ""


def find_most_similar_index(str_list, target_str):
    """
    Given a list of strings and a target string, returns the index of the most similar string in the list.
    """
    # Initialize variables to keep track of the most similar string and its index
    most_similar_str = None
    most_similar_index = 0
    highest_similarity = 0

    # Iterate through each string in the list
    for i, str in enumerate(str_list):
        # Calculate the similarity between the current string and the target string
        similarity = str_similarity(str, target_str)

        # If the current string is more similar than the previous most similar string, update the variables
        if similarity > highest_similarity:
            most_similar_str = str
            most_similar_index = i
            highest_similarity = similarity

    # Return the index of the most similar string
    return most_similar_index


def str_similarity(str1, str2):
    seq = difflib.SequenceMatcher(None, str1, str2)
    return seq.ratio()


def judge_multi_choice(choices, answer, response, alphas=None):
    response = response.lower()
    if response.split("\n\n")[0] in [chr(ord("a") + i) for i in range(len(choices))]:
        response = response.split("\n\n")[0]
    elif response.split("\n\n")[-1].split(".")[0] in [chr(ord("a") + i) for i in range(len(choices))]:
        response = response.split("\n\n")[-1].split(".")[0]

    response = parse_response(response)
    alphas = [chr(ord("a") + i) for i in range(len(choices))]
    choices = [choice.lower() for choice in choices]
    flag = False
    response = response.strip().lower()
    response = response.replace("\n", "")
    split_response = response.split(".")[0]
    split_response = split_response.split(":")[-1]
    answer = answer.strip().lower()

    if len(split_response) > 300:
        flag = False
    # letter,letter.  choice,choice
    if split_response == answer:
        flag = True

    # letter,choice
    elif split_response in alphas:
        if choices[ord(split_response) - ord("a")] == answer:
            flag = True

    # choice letter
    elif split_response in choices:
        if answer in alphas and split_response == choices[ord(answer) - ord("a")]:
            flag = True
    # unparsed
    else:
        index = find_most_similar_index(choices, response)
        if alphas[index] == answer or choices[index] == answer:
            flag = True
    return flag


def parse_response(response):
    response = response.lower()
    if "boxed" in response:
        response = extract_boxed_content(response)
    elif "<answer>" in response:
        response = extract(response, "answer")
    answer_patterns = [
        "**answer**:",
        "**answer**",
        "*answer*:",
        "**answer:**",
        "answer is",
        "answer:",
        "答案:",
        "final answer",
        "final answer is",
    ]
    for answer_pattern in answer_patterns:
        if answer_pattern in response:
            response = response.split(answer_pattern)[-1]
    return response


answer = "C"

In [26]:
text = "**E**\n\n**Explanation:**\n\nBased on the provided images (MRI and CT scans), the most likely diagnosis is **tricuspid stenosis**. Here\u2019s why:\n\n*   **MRI Findings:** The MRI shows significant enlargement of the right atrium (RA) and right ventricle (RV). This is a classic sign of increased pressure in the right side of the heart, which is often caused by a narrowed tricuspid valve.\n*   **CT Scan Findings:** The CT scan confirms the enlarged right atrium and suggests a possible mass or thickening in the right atrium, further supporting the tricuspid stenosis.\n\nLet's look at why the other options are less likely:\n\n*   **(A) Pulmonary hypertension:** While heart failure can be associated with pulmonary hypertension, the MRI findings point more specifically to a structural valve problem.\n*   **(B) Hypertrophic cardiomyopathy:** This typically presents with symptoms related to outflow obstruction, not primarily right-sided heart enlargement.\n"
options = {
    "A": "Pulmonary hypertension",
    "B": "Hypertrophic cardiomyopathy",
    "C": "Constrictive pericarditis",
    "D": "Restrictive cardiomyopathy",
    "E": "Tricuspid stenosis",
}

choices = []
for option, label in options.items():
    choice = f"{option}. {label}"
    choices.append(choice)

response = extract(text, "answer")
response

correct = judge_multi_choice(choices, answer, response)

In [27]:
choices

['A. Pulmonary hypertension',
 'B. Hypertrophic cardiomyopathy',
 'C. Constrictive pericarditis',
 'D. Restrictive cardiomyopathy',
 'E. Tricuspid stenosis']

In [28]:
correct

True